### Polish with Bielik 7B Instruct v0.1 GPTQ (quantized)
I've made a bonus file here for these sentence generation tests because loading more than one model in a notebook just stores it in memory I guess and makes it so that further models can run into weird errors.

In [1]:
# %%capture
# 
# !pip install accelerate -q
# !pip install optimum -q
# !pip install auto-gptq -q
# !pip install ctransformers[cuda]

In [2]:
import gc
import torch
import warnings

#warnings.filterwarnings("ignore") # good for demo, not good idea for production :)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ" #https://colab.research.google.com/drive/1Al9glPVCuOXbtDsks8cMcuzkuu8YDzpg?usp=sharing#scrollTo=LDo7PEL-gyyy
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

CUDA extension not installed.
CUDA extension not installed.
C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [4]:
human_prompt = "Napisz zdaniez, w którym znajduje się słowo 'taniec'"
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1, #0.1 is the default, changing it to .5 leads to... German xD
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<s>[INST]Napisz zdaniez, w którym znajduje się słowo 'taniec'[/INST]  Taniec to jedna z najstarszych form sztuki, który łączy w sobie elementy muzyki, ruchu i wyrażania emocji.


In [5]:
human_prompt = ("Napisz zdanie, w którym pojawia się słowo 'tańczący'.")
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz zdanie, w którym pojawia się słowo 'tańczący'.[/INST]  Wśród tańczących na parkiecie, tylko ona poruszała się z taką gracją i lekkością.


In [6]:
human_prompt = ("Napisz 40 zdania, w ktorych się pojawia słowo 'taniec'.")
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=40000, #256 is the default
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz 40 zdania, w ktorych się pojawia słowo 'taniec'.[/INST]  "Taniec jest jak muzyka duszy, wyraża emocje i pozwala na chwilę wolności. W weekend zorganizowaliśmy warsztaty tańca nowoczesnego, które były pełne energii i kreatywności. Na parkiecie hotelowym, podczas studniówki, wszyscy bawili się wspaniale, tworząc niezapomniane wspomnienia. W sobotę wieczorem, na plaży, przy dźwiękach fal, odbył się pokaz tańca towarzyskiego, który był prawdziwym spektaklem. Nauka tańca w grupie dodaje odwagi i motywacji, a każde spotkanie to fascynująca podróż w świat rytmu i muzyki. W niedzielne popołudnie, w parku miejskim, odbył się festiwal tańca ulicznego, który przyciągnął tłumy widzów i wykonawców."


### Sample output text -- one sentence missing "taniec/dance" equivalent, formatted with newlines for easier reading

"Taniec jest jak muzyka duszy, wyraża emocje i pozwala na chwilę wolności. 
W weekend zorganizowaliśmy warsztaty tańca nowoczesnego, które były pełne energii i kreatywności. 
Na parkiecie hotelowym, podczas studniówki, wszyscy bawili się wspaniale, tworząc niezapomniane wspomnienia. 
W sobotę wieczorem, na plaży, przy dźwiękach fal, odbył się pokaz tańca towarzyskiego, który był prawdziwym spektaklem. 
Nauka tańca w grupie dodaje odwagi i motywacji, a każde spotkanie to fascynująca podróż w świat rytmu i muzyki. 
W niedzielne popołudnie, w parku miejskim, odbył się festiwal tańca ulicznego, który przyciągnął tłumy widzów i wykonawców."

In [7]:
human_prompt = ("Napisz 40 zdania, w ktorych się pojawia słowo 'tańczyć'.")
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=100000, #256 is the default
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz 40 zdania, w ktorych się pojawia słowo 'tańczyć'.[/INST]  Oto 40 zdań, w których pojawia się słowo "tańczyć":

1. Ona potrafi pięknie tańczyć, a jej ruchy na parkiecie hipnotyzują wszystkich wokół.
2. W sobotni wieczór miasto tętniło muzyką, a ludzie ochoczo wyszli na ulice, by tańczyć.
3. Taniec to jej pasja, poświęca wiele czasu na doskonalenie swoich umiejętności i występy.
4. Na balu wszyscy tańczyli w rytm muzyki, ciesząc się każdym krokiem i chwilą spędzoną razem.
5. On zawsze potrafi dostosować swoje ruchy do muzyki, wplatając w taniec swoją osobowość.
6. Podczas jogi, uczestnicy uczą się synchronizować swoje ruchy z oddechem, co pomaga w medytacyjnym doświadczeniu tańca.
7. Jej ulubiony styl tańca to salsa, dzięki której może wyrazić swoją osobowość i energię.
8. Na scenie tancerze tworzą żywe dzieła sztuki, ich ruchy i wyrazy twarzy opowiadają historie bez słów.
9. W szkole tańca dzieci uczą się podstawowych kroków i układów choreograficznych, rozwijając swoją 

### Sample output text -- wow, it took 18 minutes but it did the thing!

Oto 40 zdań, w których pojawia się słowo "tańczyć":

1. Ona potrafi pięknie tańczyć, a jej ruchy na parkiecie hipnotyzują wszystkich wokół.
2. W sobotni wieczór miasto tętniło muzyką, a ludzie ochoczo wyszli na ulice, by tańczyć.
3. Taniec to jej pasja, poświęca wiele czasu na doskonalenie swoich umiejętności i występy.
4. Na balu wszyscy tańczyli w rytm muzyki, ciesząc się każdym krokiem i chwilą spędzoną razem.
5. On zawsze potrafi dostosować swoje ruchy do muzyki, wplatając w taniec swoją osobowość.
6. Podczas jogi, uczestnicy uczą się synchronizować swoje ruchy z oddechem, co pomaga w medytacyjnym doświadczeniu tańca.
7. Jej ulubiony styl tańca to salsa, dzięki której może wyrazić swoją osobowość i energię.
8. Na scenie tancerze tworzą żywe dzieła sztuki, ich ruchy i wyrazy twarzy opowiadają historie bez słów.
9. W szkole tańca dzieci uczą się podstawowych kroków i układów choreograficznych, rozwijając swoją koordynację i miłość do ruchu.
10. Taniec to sposób na wyrażenie emocji i uwolnienie się od codziennych trosk.
11. Na turnieju tańca nowoczesnego każda formacja prezentuje swoje umiejętności, rywalizując z innymi zespołami.
12. W trakcie warsztatów tanecznych uczestnicy uczą się nowych stylów i technik, rozwijając swoje umiejętności i kreatywność.
13. Taniec może być terapeutyczny, pomagając w uwolnieniu emocji i redukcji stresu.
14. Na pokazie mody, projektanci i modele wspólnie tańczą, tworząc widowisko, które podkreśla kolekcję i jej przesłanie.
15. W teledyskach tancerze wcielają się w różne role, opowiadając historie piosenek za pomocą swojego ciała.
16. Na festiwalu tańca, różne zespoły i soliści prezentują swoje umiejętności w różnorodnych stylach, tworząc widowisko pełne kolorów i emocji.
17. Taniec może łączyć ludzi, tworząc wspólnotę i więzi poprzez wspólne doświadczenia taneczne.
18. Na lekcjach tańca instruktorzy uczą uczniów nie tylko konkretnych kroków, ale także ekspresji i interpretacji muzyki.
19. Taniec może być sposobem na poznanie nowych ludzi i zawarcie przyjaźni, poprzez uczestnictwo w zajęciach grupowych lub wydarzeniach tanecznych.
20. W filmach i serialach, taniec często służy jako środek wyrazu, podkreślając emocje postaci i budując napięcie w fabule.
21. Na turniejach tańca towarzyskiego pary rywalizują w różnych stylach, prezentując swoje umiejętności i grację.
22. W przedstawieniach teatralnych i baletowych, taniec jest integralną częścią fabuły i opowieści, tworząc napięcie i emocje.
23. Na imprezach okolicznościowych, goście chętnie tańczą, tworząc radosną atmosferę i okazję do integracji.
24. W sztukach walki, taniec może być elementem treningu, uczącym rytmiki, koordynacji i lekkości ruchów.
25. Na zajęciach jogi, uczestnicy uczą się synchronizować ruchy z oddechem, tworząc połączenie między ciałem a umysłem.
26. W filmach muzycznych, taniec jest ważnym elementem fabuły, odzwierciedlającym emocje bohaterów i atmosferę miejsca.
27. Na festiwalach ulicznych, różne zespoły taneczne prezentują swoje umiejętności w różnorodnych stylach, przyciągając widzów i tworząc atmosferę święta.
28. W programach telewizyjnych, taniec może być formą rywalizacji, w której uczestnicy prezentują swoje umiejętności i otrzymują oceny od jurorów.
29. Na weselach, goście chętnie tańczą, tworząc radosną atmosferę i okazję do świętowania nowożeńców.
30. W spektaklach teatralnych i baletowych, tancerze wcielają się w różne role, tworząc widowisko pełne ekspresji i emocji.
31. Na warsztatach tanecznych, uczestnicy uczą się nowych stylów i technik, rozwijając swoją kreatywność i umiejętności.
32. W filmach dokumentalnych, taniec może być sposobem na opowiedzenie historii, pokazując kulturę i tradycje różnych społeczności.
33. Na turniejach tańca sportowego, zawodnicy rywalizują w różnych kategoriach wagowych i stylach, prezentując swoje umiejętności i techniki.
34. W parkach miejskich, specjalne strefy do tańca przyciągają miłośników ruchu i muzyki, tworząc przestrzeń do kreatywnego wyrażania się.
35. Na lekcjach wychowania fizycznego, uczniowie uczą się podstawowych kroków i układów tanecznych, rozwijając swoją koordynację i umiejętność pracy w grupie.
36. W filmach muzycznych, taniec jest ważnym elementem fabuły, odzwierciedlającym emocje bohaterów i atmosferę miejsca.
37. Na festiwalach ulicznych, różne zespoły taneczne prezentują swoje umiejętności w różnorodnych stylach, przyciągając widzów i tworząc atmosferę święta.
38. W programach telewizyjnych, taniec może być formą rywalizacji, w której uczestnicy prezentują swoje umiejętności i otrzymują oceny od jurorów.
39. Na weselach, goście chętnie tańczą, tworząc radosną atmosferę i okazję do świętowania nowożeńców.
40. W spektaklach teatralnych i baletowych, tancerze wcielają się w różne role, tworząc widowisko pełne ekspresji i emocji.